In [ ]:
#!pip install gtfs_functions
#!pip install KeplerGl
#!pip install keplergl jupyter
#!jupyter nbextension install --py --symlink --sys-prefix keplergl
#!jupyter nbextension enable --py --sys-prefix keplergl
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension
#!pip install gtfs_functions matplotlib basemap
#!pip install geopy
#!pip install pyshp
#!pip install partridge
#!pip install fiona
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
import gtfs_functions as gtfs
from gtfs_functions import Feed
import networkx as nx
import geopandas as gpd
import folium
from shapely.geometry import Point, LineString, Polygon, shape
from folium.plugins import MarkerCluster
#import keplergl as kp
from google.colab import output
from IPython.display import display
import pandas as pd
from gtfs_functions.gtfs_plots import map_gdf
import plotly.graph_objs as go
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from geopy.distance import geodesic
from math import radians, sin, cos, sqrt, atan2
from itertools import combinations
import numpy as np
import shapefile as shp
import seaborn as sns
import os
import copy
import fiona
import csv
import math
import colorsys
from difflib import SequenceMatcher
from folium.plugins import HeatMap
import branca.colormap as cmb
import re
from collections import defaultdict
import partridge as ptg
from shapely.ops import nearest_points
from scipy.spatial import cKDTree
from shapely.geometry import mapping
import heapq
from folium.features import DivIcon

#####BKK

In [ ]:
feed_bkk = Feed("budapest_gtfs.zip", time_windows=[6, 7]) #itt változtathatunk az időintervallumon,
#feed = Feed("budapest_gtfs.zip")
routes_bkk = feed_bkk.routes
trips_bkk = feed_bkk.trips
stops_bkk = feed_bkk.stops
stop_times_bkk = feed_bkk.stop_times
shapes_bkk = feed_bkk.shapes

In [ ]:
segments_gdf_bkk = feed_bkk.segments
segments_gdf_bkk.head(2)
#segments_gdf_bkk.info()

In [ ]:
segments_freq_bkk = feed_bkk.segments_freq
segments_freq_bkk.head(2)
#print(type(segments_freq))

In [ ]:
colnames=['route_name', 'capacity']
df = pd.read_csv('bkk_capacity.csv', names=colnames, header=None)
df.head(2)

In [ ]:
segments_freq_bkk = segments_freq_bkk.merge(df, on='route_name')
segments_freq_bkk.head(2)

In [ ]:
segments_freq_bkk = segments_freq_bkk.drop(columns = ['route_id','segment_name','start_stop_name','end_stop_name','window','min_per_trip','segment_id'])
segments_freq_bkk.head(2)

###VOLÁN

In [ ]:
# Read the feed for the specified view
feed = ptg.load_feed('volanbusz_gtfs.zip')

# Access different components of the feed
routes_volan = feed.routes
trips_volan = feed.trips
stops_volan = feed.stops
stop_times_volan = feed.stop_times
shapes_volan = feed.shapes

In [ ]:
# Convert time to seconds since midnight
def time_to_seconds(t):
    try:
        # Handle numeric times directly (e.g., 23100.0 seconds)
        if isinstance(t, (int, float)):
            return int(t)
        # Handle HH:MM:SS format
        h, m, s = map(int, t.split(':'))
        return h * 3600 + m * 60 + s
    except Exception as e:
        return None  # Return None if there is an issue with conversion

In [ ]:
# Apply the conversion function to the departure_time column
stop_times_volan['time_in_seconds'] = stop_times_volan['departure_time'].apply(time_to_seconds)

# Define your desired time window in seconds
start_time = time_to_seconds('06:00:00')
end_time = time_to_seconds('07:00:00')

# Filter the stop_times DataFrame based on the time window
stop_times_volan = stop_times_volan[
    (stop_times_volan['time_in_seconds'] >= start_time) &
    (stop_times_volan['time_in_seconds'] <= end_time)
]

# Drop the helper column if you don't need it anymore
stop_times_volan = stop_times_volan.drop(columns=['time_in_seconds'])

In [ ]:
def create_segments(stop_times, stops, trips, routes, shapes):
    # Merge stop_times with stops to get stop coordinates
    stop_times = stop_times.merge(stops[['stop_id', 'stop_lat', 'stop_lon']], on='stop_id')

    # Merge stop_times with trips to get trip and route information
    stop_times = stop_times.merge(trips[['trip_id', 'route_id', 'direction_id', 'shape_id']], on='trip_id')

    # Merge stop_times with routes to get route_name
    stop_times = stop_times.merge(routes[['route_id', 'route_short_name']], on='route_id')

    # Sort stop_times by trip_id and stop_sequence
    stop_times = stop_times.sort_values(['trip_id', 'stop_sequence'])

    # Prepare shapes data
    shapes = shapes.sort_values(['shape_id', 'shape_pt_sequence'])
    shapes_grouped = shapes.groupby('shape_id')

    segments = []
    for trip_id, group in stop_times.groupby('trip_id'):
        shape_id = group.iloc[0]['shape_id']
        if shape_id in shapes_grouped.groups:
            shape_points = shapes_grouped.get_group(shape_id)[['shape_pt_lon', 'shape_pt_lat']].values
            for i in range(len(group) - 1):
                from_stop = group.iloc[i]
                to_stop = group.iloc[i + 1]
                from_index = from_stop['stop_sequence']
                to_index = to_stop['stop_sequence']
                segment_shape_points = shape_points[from_index:to_index + 1]

                if len(segment_shape_points) < 2:
                    segment_shape_points = [(from_stop['stop_lon'], from_stop['stop_lat']),
                                            (to_stop['stop_lon'], to_stop['stop_lat'])]

                segment = {
                    'trip_id': trip_id,
                    'route_id': from_stop['route_id'],
                    'route_short_name': from_stop['route_short_name'],
                    'direction_id': from_stop['direction_id'],
                    'from_stop_id': from_stop['stop_id'],
                    'to_stop_id': to_stop['stop_id'],
                    'geometry': LineString(segment_shape_points)
                }
                segments.append(segment)

    segments_df = pd.DataFrame(segments)
    return gpd.GeoDataFrame(segments_df, geometry='geometry')

In [ ]:
segments_gdf_volan = create_segments(stop_times_volan, stops_volan, trips_volan, routes_volan, shapes_volan)

In [ ]:
def create_segments_freq(segments_gdf):
    # Group by from_stop_id, to_stop_id, route_id, route_short_name, direction_id and count the occurrences
    segments_freq = segments_gdf.groupby(['from_stop_id', 'to_stop_id', 'route_id', 'route_short_name', 'direction_id']).size().reset_index(name='frequency')

    # Merge with the segments to get the geometries
    segments_freq = segments_freq.merge(segments_gdf[['from_stop_id', 'to_stop_id', 'route_id', 'route_short_name', 'direction_id', 'geometry']],
                                        on=['from_stop_id', 'to_stop_id', 'route_id', 'route_short_name', 'direction_id']).drop_duplicates()

    # Rename columns as requested
    segments_freq = segments_freq.rename(columns={
        'from_stop_id': 'start_stop_id',
        'to_stop_id': 'end_stop_id',
        'route_short_name': 'route_name',
        'frequency': 'ntrips'
    })

    segments_freq = segments_freq.drop(columns=['route_id'])

    return gpd.GeoDataFrame(segments_freq, geometry='geometry')

In [ ]:
# Create segment frequencies GeoDataFrame
segments_freq_volan = create_segments_freq(segments_gdf_volan)

# Display segment frequencies
#print(segments_freq_volan.head())

In [ ]:
colnames=['route_name', 'capacity']
df = pd.read_csv('volan_capacity.csv', names=colnames, header=None)

df['route_name'] = df['route_name'].astype(str)

# Extract route names and capacity values
route_names = df['route_name'].tolist()
capacity_dict = dict(zip(df['route_name'], df['capacity']))

In [ ]:
def filter_segment_freq(segments_freq,route_names,capacity_dict):
  segments_freq['route_name'] = segments_freq['route_name'].astype(str).str.strip()

  # Filter the GeoDataFrame based on route names
  segments_freq = segments_freq[segments_freq['route_name'].isin(route_names)]
  segments_freq['capacity'] = segments_freq['route_name'].map(capacity_dict)
  # Ensure the CRS for the geometry GeoDataFrame if not set
  if segments_freq.crs is None:
      segments_freq.set_crs(epsg=4326, inplace=True)

  # Define the path to the shapefile
  shapefile_path = 'zone_zone.SHP'

  # Check if the shapefile exists
  if not os.path.exists(shapefile_path):
      raise FileNotFoundError(f"Shapefile not found: {shapefile_path}")

  # Load the shapefile using Fiona
  with fiona.open(shapefile_path) as shapefile:
      # Ensure shapefile has a CRS
      if shapefile.crs is None:
          # Replace 'EPSG:4326' with the correct CRS if known
          shapefile_crs = 'EPSG:4326'
      else:
          shapefile_crs = shapefile.crs

      # Convert the shapefile features to GeoDataFrame
      shapefile_gdf = gpd.GeoDataFrame.from_features(shapefile, crs=shapefile_crs)

      # Filter the shapefile GeoDataFrame based on the 'NO' attribute
      shapefile_filtered = shapefile_gdf[(shapefile_gdf['NO'] >= 1000) & (shapefile_gdf['NO'] <= 23999)]
      #shapefile_filtered = shapefile_gdf

      # Ensure CRS match
      if shapefile_filtered.crs != segments_freq.crs:
          shapefile_filtered = shapefile_filtered.to_crs(segments_freq.crs)

      # Convert filtered shapes to Shapely geometries
      filtered_shapes = [shape(geom_shape) for geom_shape in shapefile_filtered.geometry]

      # Function to check if a point is within any of the city shapes
      def point_in_city_shapes(point):
          return any(city_shape.contains(point) for city_shape in filtered_shapes)

      # Create masks to filter rows
      mask_first_in_city = segments_freq.geometry.apply(lambda line: point_in_city_shapes(Point(line.coords[0])))
      mask_last_in_city = segments_freq.geometry.apply(lambda line: point_in_city_shapes(Point(line.coords[-1])))

      # Filter rows where either the first or last point is not in the city shapes
      segments_freq = segments_freq[mask_first_in_city & mask_last_in_city]

      return segments_freq

In [ ]:
segments_freq_volan = filter_segment_freq(segments_freq_volan, route_names, capacity_dict)

###MÁV

In [ ]:
# Read the feed for the specified view
feed_mav = ptg.load_feed('mav_gtfs.zip')

# Access different components of the feed
routes_mav = feed_mav.routes
trips_mav = feed_mav.trips
stops_mav = feed_mav.stops
stop_times_mav = feed_mav.stop_times
shapes_mav = feed_mav.shapes

In [ ]:
# Apply the conversion function to the departure_time column
stop_times_mav['time_in_seconds'] = stop_times_mav['departure_time'].apply(time_to_seconds)

# Define your desired time window in seconds
start_time = time_to_seconds('06:00:00')
end_time = time_to_seconds('07:00:00')

# Filter the stop_times DataFrame based on the time window
stop_times_mav = stop_times_mav[
    (stop_times_mav['time_in_seconds'] >= start_time) &
    (stop_times_mav['time_in_seconds'] <= end_time)
]

# Drop the helper column if you don't need it anymore
stop_times_mav = stop_times_mav.drop(columns=['time_in_seconds'])

In [ ]:
def create_segments2(stop_times, stops, trips, routes, shapes):
    # Merge stop_times with stops to get stop coordinates
    stop_times = stop_times.merge(stops[['stop_id', 'stop_lat', 'stop_lon']], on='stop_id')

    # Merge stop_times with trips to get trip and route information
    stop_times = stop_times.merge(trips[['trip_id', 'route_id', 'shape_id']], on='trip_id')

    # Merge stop_times with routes to get route_name
    stop_times = stop_times.merge(routes[['route_id', 'route_short_name']], on='route_id')

    # Sort stop_times by trip_id and stop_sequence
    stop_times = stop_times.sort_values(['trip_id', 'stop_sequence'])

    # Prepare shapes data
    shapes = shapes.sort_values(['shape_id', 'shape_pt_sequence'])
    shapes_grouped = shapes.groupby('shape_id')

    segments = []
    for trip_id, group in stop_times.groupby('trip_id'):
        shape_id = group.iloc[0]['shape_id']
        if shape_id in shapes_grouped.groups:
            shape_points = shapes_grouped.get_group(shape_id)[['shape_pt_lon', 'shape_pt_lat']].values.tolist()
            for i in range(len(group) - 1):
                from_stop = group.iloc[i]
                to_stop = group.iloc[i + 1]

                # Find the indices of the stops in the shape points
                from_index = next((idx for idx, point in enumerate(shape_points) if (point[0], point[1]) == (from_stop['stop_lon'], from_stop['stop_lat'])), None)
                to_index = next((idx for idx, point in enumerate(shape_points) if (point[0], point[1]) == (to_stop['stop_lon'], to_stop['stop_lat'])), None)

                # If either index is not found, default to direct line between stops
                if from_index is None or to_index is None or from_index >= to_index:
                    segment_shape_points = [(from_stop['stop_lon'], from_stop['stop_lat']),
                                            (to_stop['stop_lon'], to_stop['stop_lat'])]
                else:
                    segment_shape_points = shape_points[from_index:to_index + 1]

                segment = {
                    'trip_id': trip_id,
                    'route_id': from_stop['route_id'],
                    'route_short_name': from_stop['route_short_name'],
                    'from_stop_id': from_stop['stop_id'],
                    'to_stop_id': to_stop['stop_id'],
                    'geometry': LineString(segment_shape_points)
                }
                segments.append(segment)

    segments_df = pd.DataFrame(segments)
    return gpd.GeoDataFrame(segments_df, geometry='geometry')

In [ ]:
def create_segments3(stop_times, stops, trips, routes, shapes, decimal_places=6):
    # Round coordinates in stops and shapes to the same number of decimal places
    stops['stop_lat'] = stops['stop_lat'].round(decimal_places)
    stops['stop_lon'] = stops['stop_lon'].round(decimal_places)
    shapes['shape_pt_lat'] = shapes['shape_pt_lat'].round(decimal_places)
    shapes['shape_pt_lon'] = shapes['shape_pt_lon'].round(decimal_places)
    # Merge stop_times with stops to get stop coordinates
    stop_times = stop_times.merge(stops[['stop_id', 'stop_lat', 'stop_lon']], on='stop_id')

    # Merge stop_times with trips to get trip and route information
    stop_times = stop_times.merge(trips[['trip_id', 'route_id', 'shape_id']], on='trip_id')

    # Merge stop_times with routes to get route_name
    stop_times = stop_times.merge(routes[['route_id', 'route_short_name']], on='route_id')

    # Sort stop_times by trip_id and stop_sequence
    stop_times = stop_times.sort_values(['trip_id', 'stop_sequence'])

    # Prepare shapes data
    shapes = shapes.sort_values(['shape_id', 'shape_pt_sequence'])
    shapes_grouped = shapes.groupby('shape_id')

    segments = []
    for trip_id, group in stop_times.groupby('trip_id'):
        shape_id = group.iloc[0]['shape_id']
        if shape_id in shapes_grouped.groups:
            shape_points = shapes_grouped.get_group(shape_id)[['shape_pt_lon', 'shape_pt_lat']].values.tolist()
            for i in range(len(group) - 1):
                from_stop = group.iloc[i]
                to_stop = group.iloc[i + 1]

                # Find the indices of the stops in the shape points
                from_index = next((idx for idx, point in enumerate(shape_points) if (point[0], point[1]) == (from_stop['stop_lon'], from_stop['stop_lat'])), None)
                to_index = next((idx for idx, point in enumerate(shape_points) if (point[0], point[1]) == (to_stop['stop_lon'], to_stop['stop_lat'])), None)

                # If either index is not found, default to direct line between stops
                if from_index is None or to_index is None or from_index >= to_index:
                    segment_shape_points = [(from_stop['stop_lon'], from_stop['stop_lat']),
                                            (to_stop['stop_lon'], to_stop['stop_lat'])]
                else:
                    segment_shape_points = shape_points[from_index:to_index + 1]

                # Ensure segment_shape_points has more than 2 points for the LineString
                if len(segment_shape_points) < 2:
                    segment_shape_points = [(from_stop['stop_lon'], from_stop['stop_lat']),
                                            (to_stop['stop_lon'], to_stop['stop_lat'])]

                segment = {
                    'trip_id': trip_id,
                    'route_id': from_stop['route_id'],
                    'route_short_name': from_stop['route_short_name'],
                    'from_stop_id': from_stop['stop_id'],
                    'to_stop_id': to_stop['stop_id'],
                    'geometry': LineString(segment_shape_points)
                }
                segments.append(segment)

    segments_df = pd.DataFrame(segments)
    return gpd.GeoDataFrame(segments_df, geometry='geometry')

In [ ]:
segments_mav = create_segments(stop_times_mav, stops_mav, trips_mav, routes_mav, shapes_mav)

In [ ]:
segments_mav = create_segments2(stop_times_mav, stops_mav, trips_mav, routes_mav, shapes_mav)

In [ ]:
segments_mav['direction_id'] = 0
segments_mav.head()

In [ ]:
segments_freq_mav = create_segments_freq(segments_mav)
#segments_freq_mav = create_segments_freq2(segments_mav,stop_times_mav)
segments_freq_mav.head()

In [ ]:
colnames=['route_name', 'capacity']
df2 = pd.read_csv('mav_capacity.csv', names=colnames, header=None)

df2['route_name'] = df2['route_name'].astype(str)

# Extract route names and capacity values
route_names2 = df2['route_name'].tolist()
capacity_dict2 = dict(zip(df2['route_name'], df2['capacity']))

In [ ]:
segments_freq_mav = filter_segment_freq(segments_freq_mav, route_names2, capacity_dict2)

In [ ]:
original_crs = 'EPSG:4326'
if segments_freq_bkk.crs is None:
    segments_freq_bkk = segments_freq_bkk.set_crs(original_crs)

if segments_freq_volan.crs is None:
    segments_freq_volan = segments_freq_volan.set_crs(original_crs)

if segments_freq_mav.crs is None:
    segments_freq_mav = segments_freq_mav.set_crs(original_crs)

# Now you can transform to the desired CRS
desired_crs = 'EPSG:4326'
segments_freq_bkk = segments_freq_bkk.to_crs(desired_crs)
segments_freq_volan = segments_freq_volan.to_crs(desired_crs)
segments_freq_mav = segments_freq_mav.to_crs(desired_crs)

# Concatenate the GeoDataFrames
segments_freq = pd.concat([segments_freq_bkk, segments_freq_volan, segments_freq_mav], ignore_index=True)

# Ensure the result is still a GeoDataFrame
segments_freq = gpd.GeoDataFrame(segments_freq, geometry='geometry')

###Joint heatmap

In [ ]:
seq_copy = copy.deepcopy(segments_freq_bkk)
seq_copy['weight'] = seq_copy['ntrips'] * seq_copy['capacity']

# Step 2: Normalize the weights for visualization
seq_copy['normalized_weight'] = seq_copy['weight'] / seq_copy['weight'].max()

# Step 3: Create a folium map centered on the mean coordinates of the data
mean_x, mean_y = seq_copy.geometry.centroid.x.mean(), seq_copy.geometry.centroid.y.mean()
m = folium.Map(location=[mean_y, mean_x], zoom_start=12)

# Step 4: Add the lines to the map
colormap = cmb.LinearColormap(colors=['blue', 'green', 'yellow', 'red'], vmin=0, vmax=1)
for _, row in seq_copy.iterrows():
    weight = row['normalized_weight']
    # Extracting coordinates directly
    locations = [(coord[1], coord[0]) for coord in row['geometry'].coords]
    folium.PolyLine(
        locations=locations,
        weight=weight * 10 + 0.1,  # Adjust thickness
        color=colormap(weight),  # Color based on weight
        opacity=0.7
    ).add_to(m)

# Step 5: Add colormap to the map
colormap.caption = 'Heatmap Intensity'
m.add_child(colormap)
#m.save('heatmap.html')
m

###Multigraph

In [ ]:
def add_edges(G, gdf):
    for idx, row in gdf.iterrows():
        # Extract start and end coordinates
        start_coords = (row['geometry'].coords[0][1], row['geometry'].coords[0][0])
        end_coords = (row['geometry'].coords[-1][1], row['geometry'].coords[-1][0])

        # Calculate the product of ntrips and capacity
        ntrips_capacity = row['ntrips'] * row['capacity']

        # Create a unique key for the edge
        key = f"{row['route_name']}_{ntrips_capacity}"

        # Add edge to the graph with specified keys and additional attributes
        if row['direction_id'] == 0:
          G.add_edge(start_coords, end_coords, key=key, route_name=row['route_name'], ntrips_capacity=ntrips_capacity)
        elif row['direction_id'] == 1:
          G.add_edge(end_coords, start_coords, key=key, route_name=row['route_name'], ntrips_capacity=ntrips_capacity)

In [ ]:
G = nx.MultiDiGraph()
# Add edges from segments_freq to the graph
add_edges(G, segments_freq)

# Display basic information about the graph
#print(info(G))

In [ ]:
def create_folium_map(G):
    # Create a folium map centered around Budapest
    m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)

    # Add edges to the folium map with tooltips
    for u, v, data in G.edges(data=True):
        # Create the coordinates list for the PolyLine
        coords = [u, v]

        # Create the tooltip text
        tooltip_text = f"Route: {data['route_name']}<br>Capacity: {data['ntrips_capacity']}"

        # Add the PolyLine to the map with the tooltip
        folium.PolyLine(coords, color='blue', weight=2.5, opacity=1, tooltip=tooltip_text).add_to(m)

    # Add nodes to the folium map
    for node in G.nodes():
        folium.CircleMarker(location=node, radius=2, color='red').add_to(m)

    return m

In [ ]:
def create_folium_map_multicolor(G):
    # Create a folium map centered around Budapest
    m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)

    # Generate a list of distinct colors
    colors = list(mcolors.CSS4_COLORS.values())
    max_colors = min(len(colors), 50)  # Limit to 30 distinct colors if there are fewer colors available
    colors = colors[:max_colors]

    # Create a dictionary to track the number of parallel edges
    edge_count = {}

    # Add edges to the folium map with tooltips
    for u, v, key, data in G.edges(keys=True, data=True):
        # Create the coordinates list for the PolyLine
        coords = [u, v]

        # Create a unique identifier for the edge pair (u, v)
        edge_id = (u, v)
        if edge_id not in edge_count:
            edge_count[edge_id] = 0
        edge_index = edge_count[edge_id]
        edge_count[edge_id] += 1

        # Assign a color based on the edge index
        color = colors[edge_index % len(colors)]

        # Create the tooltip text
        tooltip_text = f"Route: {data['route_name']}<br>Capacity: {data['ntrips_capacity']}"

        # Add the PolyLine to the map with the tooltip
        folium.PolyLine(coords, color=color, weight=2.5, opacity=1, tooltip=tooltip_text).add_to(m)

    # Add nodes to the folium map
    for node in G.nodes():
        folium.CircleMarker(location=node, radius=2, color='red').add_to(m)

    return m

In [ ]:
s = create_folium_map(G)#_multicolor(G)

# Display the map
#m.save("multidigraph_map.html")
s

###Normal graph

In [ ]:
def add_edges_with_weights(G, gdf):
    for idx, row in gdf.iterrows():
        # Extract start and end coordinates
        start_coords = (row['geometry'].coords[0][1], row['geometry'].coords[0][0])
        end_coords = (row['geometry'].coords[-1][1], row['geometry'].coords[-1][0])

        ntrips_capacity = row['ntrips'] * row['capacity']

        # Add edge to the graph with ntrips as weight
        if row['direction_id'] == 0:
          G.add_edge(start_coords, end_coords, weight=ntrips_capacity, attr_dict=row.to_dict())
        elif row['direction_id'] == 1:
          G.add_edge(end_coords, start_coords, weight=ntrips_capacity, attr_dict=row.to_dict())

In [ ]:
def add_edges_with_weights2(G, gdf):
    for idx, row in gdf.iterrows():
        # Extract start and end coordinates
        start_coords = (row['geometry'].coords[0][1], row['geometry'].coords[0][0])
        end_coords = (row['geometry'].coords[-1][1], row['geometry'].coords[-1][0])

        ntrips_capacity = row['ntrips'] * row['capacity']

        if row['direction_id'] == 0:
            if G.has_edge(start_coords, end_coords):
                # If the edge exists, update the weight
                G[start_coords][end_coords]['weight'] += ntrips_capacity
                G[start_coords][end_coords]['ntrips_capacity'] += ntrips_capacity
            else:
                # If the edge does not exist, add it with the new weight and attributes
                G.add_edge(start_coords, end_coords, weight=ntrips_capacity, route_name=row['route_name'], ntrips_capacity=ntrips_capacity)
        elif row['direction_id'] == 1:
            if G.has_edge(end_coords, start_coords):
                # If the edge exists, update the weight
                G[end_coords][start_coords]['weight'] += ntrips_capacity
                G[end_coords][start_coords]['ntrips_capacity'] += ntrips_capacity
            else:
                # If the edge does not exist, add it with the new weight and attributes
                G.add_edge(end_coords, start_coords, weight=ntrips_capacity, route_name=row['route_name'], ntrips_capacity=ntrips_capacity)

In [ ]:
G2 = nx.DiGraph()
add_edges_with_weights2(G2, segments_freq)

In [ ]:
def create_folium_map2(G):
    # Create a folium map centered around Budapest
    m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)

    # Add edges to the folium map with tooltips
    for u, v, data in G.edges(data=True):
        # Create the coordinates list for the PolyLine
        coords = [u, v]

        # Create the tooltip text
        capacity = data.get('ntrips_capacity', 'N/A')
        tooltip_text = f"Capacity: {capacity}"

        # Add the PolyLine to the map with the tooltip
        folium.PolyLine(coords, color='blue', weight=2.5, opacity=1, tooltip=tooltip_text).add_to(m)

    # Add nodes to the folium map
    for node in G.nodes():
        folium.CircleMarker(location=node, radius=2, color='red').add_to(m)

    return m

In [ ]:
def create_folium_heatmap(G):
    # Create a folium map centered around Budapest
    m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)

    # Normalize the edge weights for color scaling
    edge_weights = [data.get('ntrips_capacity', 1) for u, v, data in G.edges(data=True)]
    min_weight = min(edge_weights)
    max_weight = max(edge_weights)

    def get_color(weight):
        # Normalize the weight to the range [0, 1]
        norm_weight = (weight - min_weight) / (max_weight - min_weight) if max_weight != min_weight else 0.5
        # Get a color from the 'rainbow' colormap
        color = cm.rainbow(norm_weight)
        # Convert color to hex
        return mcolors.to_hex(color)

    # Add edges to the folium map with tooltips
    for u, v, data in G.edges(data=True):
        # Create the coordinates list for the PolyLine
        coords = [u, v]

        # Get the weight and color for the edge
        capacity = data.get('ntrips_capacity', 1)
        color = get_color(capacity)

        # Create the tooltip text
        tooltip_text = f"Capacity: {capacity}"

        # Add the PolyLine to the map with the tooltip
        folium.PolyLine(coords, color=color, weight=2.5, opacity=1, tooltip=tooltip_text).add_to(m)

    # Add nodes to the folium map
    for node in G.nodes():
        folium.CircleMarker(location=node, radius=2, color='gray').add_to(m)

    return m

In [ ]:
r = create_folium_heatmap(G2)
r

In [ ]:
r = create_folium_map2(G2)
#r.save('graf.html')
r

In [ ]:
G3 = nx.DiGraph()
add_edges_with_weights2(G3, segments_freq_mav)

In [ ]:
r = create_folium_map2(G3)
r

In [ ]:
G3.nodes()

###Combining nodes

In [ ]:
def filter_stops(stops, shapefile_path='zone_zone.SHP'):
    # Convert stops to GeoDataFrame if it's not already one
    if not isinstance(stops, gpd.GeoDataFrame):
        if 'geometry' not in stops.columns:
            raise ValueError("The stops DataFrame must have a 'geometry' column with Point geometries.")
        stops = gpd.GeoDataFrame(stops, geometry='geometry')

    # Ensure the CRS for the stops GeoDataFrame if not set
    if stops.crs is None:
        stops.set_crs(epsg=4326, inplace=True)

    # Check if the shapefile exists
    if not os.path.exists(shapefile_path):
        raise FileNotFoundError(f"Shapefile not found: {shapefile_path}")

    # Load the shapefile using Fiona
    with fiona.open(shapefile_path) as shapefile:
        # Ensure shapefile has a CRS
        if shapefile.crs is None:
            shapefile_crs = 'EPSG:4326'
        else:
            shapefile_crs = shapefile.crs

        # Convert the shapefile features to a GeoDataFrame
        shapefile_gdf = gpd.GeoDataFrame.from_features(shapefile, crs=shapefile_crs)

        # Filter the shapefile GeoDataFrame based on the 'NO' attribute
        shapefile_filtered = shapefile_gdf[(shapefile_gdf['NO'] >= 1000) & (shapefile_gdf['NO'] <= 23999)]

        # Ensure CRS match
        if shapefile_filtered.crs != stops.crs:
            shapefile_filtered = shapefile_filtered.to_crs(stops.crs)

        # Convert filtered shapes to Shapely geometries
        filtered_shapes = [shape(geom_shape) for geom_shape in shapefile_filtered.geometry]

        # Function to check if a point (stop) is within any of the city shapes
        def point_in_city_shapes(point):
            return any(city_shape.contains(point) for city_shape in filtered_shapes)

        # Create a mask to filter stops within the city shapes
        mask_in_city = stops.geometry.apply(lambda point: point_in_city_shapes(point))

        # Filter stops within the city shapes
        filtered_stops = stops[mask_in_city]

        return filtered_stops

In [ ]:
stops_bkk = stops_bkk.drop(columns = ['stop_code','location_sub_type','wheelchair_boarding'])
stops_volan = stops_volan.drop(columns = ['platform_code'])
stops_volan['geometry'] = stops_volan.apply(lambda row: Point(row['stop_lon'], row['stop_lat']), axis=1)
stops_volan = gpd.GeoDataFrame(stops_volan, geometry='geometry')
stops_volan = filter_stops(stops_volan)
stops_mav = stops_mav.drop(columns = ['stop_code','stop_desc','wheelchair_boarding','stop_timezone'])
stops_mav['geometry'] = stops_mav.apply(lambda row: Point(row['stop_lon'], row['stop_lat']), axis=1)
stops_mav = gpd.GeoDataFrame(stops_mav, geometry='geometry')
stops_mav = filter_stops(stops_mav)
stops = pd.concat([stops_bkk, stops_volan, stops_mav], ignore_index=True)

In [ ]:
# Assuming 'stops' is the GeoDataFrame and has columns 'stop_name', 'stop_id', 'geometry'
stops_list = []

# Step 1: Extract necessary data into a list of [stop_name, stop_id, coordinate]
for _, row in stops.iterrows():
    stop_name = row['stop_name']
    stop_id = row['stop_id']
    coordinate = (row['stop_lon'], row['stop_lat'])
    stops_list.append([stop_name, stop_id, coordinate])

In [ ]:
def remove_brackets_and_extras(stop_name):
    stop_name = re.sub(r'\(.*?\)', '', stop_name)         # Remove text in parentheses
    stop_name = re.sub(r'\[.*?\]', '', stop_name)         # Remove text in square brackets
    stop_name = re.sub(r'\bH\b', '', stop_name)           # Remove standalone 'H'
    stop_name = re.sub(r'\bM\b', '', stop_name)           # Remove standalone 'M'
    stop_name = stop_name.replace('+', '')                # Remove the '+' sign
    stop_name = re.sub(r'\bBudapest,\s*', '', stop_name)  # Remove 'Budapest, '
    return stop_name.strip()

In [ ]:
# Clean stop names in the stops_list
for stop in stops_list:
    stop[0] = remove_brackets_and_extras(stop[0])

In [ ]:
# Step 3: Create a dictionary with stop_name as keys and lists of [stop_id, coordinate] as values
stops_dict = defaultdict(list)
for stop_name, stop_id, coord in stops_list:
    stops_dict[stop_name].append([stop_id, coord])

In [ ]:
def haversine(coord1, coord2):
    # Radius of the Earth in meters
    R = 6371000

    lat1, lon1 = coord1
    lat2, lon2 = coord2

    # Convert latitude and longitude from degrees to radians
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    # Haversine formula
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distance in meters
    distance = R * c
    return distance

In [ ]:
# Step 4: Group by coordinates within each stop name
def group_coordinates_by_distance(entries, threshold=1000):
    groups = []
    for stop_id, coord in entries:
        added = False
        for group in groups:
            if all(haversine(coord, existing_coord[1]) < threshold for existing_coord in group):
                group.append([stop_id, coord])
                added = True
                break
        if not added:
            groups.append([[stop_id, coord]])
    return groups

# Create a list to hold all groups of stop_ids and coordinates
grouped_by_stop = []

# Group coordinates for each stop name
for stop_name, entries in stops_dict.items():
    groups = group_coordinates_by_distance(entries)
    grouped_by_stop.extend(groups)

In [ ]:
# Step 5: Calculate average coordinates for each group and replace them in the grouped list
def calculate_average_coordinates(groups):
    for group in groups:
        if not group:
            continue
        avg_lat = sum(coord[1][1] for coord in group) / len(group)
        avg_lon = sum(coord[1][0] for coord in group) / len(group)
        average_coord = (avg_lon, avg_lat)
        for item in group:
            item[1] = average_coord

# Apply the average calculation to all groups
calculate_average_coordinates(grouped_by_stop)

In [ ]:
# Create a dictionary to store stop_id as keys and average coordinates as values
average_coords_dict = {}

# Iterate through the grouped_by_stop list to fill the dictionary
for group in grouped_by_stop:
    for stop_id, avg_coord in group:
        average_coords_dict[stop_id] = avg_coord

In [ ]:
# Function to update the geometry of each row
def update_geometry(row):
    # Get the original LineString
    line = row['geometry']

    # Look up the average coordinates using the stop IDs
    start_coord = average_coords_dict.get(row['start_stop_id'])
    end_coord = average_coords_dict.get(row['end_stop_id'])

    # If coordinates are found, update the LineString
    if start_coord and end_coord:
        # Create a new LineString with updated start and end coordinates
        new_coords = [start_coord] + list(line.coords[1:-1]) + [end_coord]
        return LineString(new_coords)
    else:
        # If not found, return the original LineString
        return line

seq_copy = copy.deepcopy(segments_freq)
seq_copy['geometry'] = seq_copy.apply(update_geometry, axis=1)

In [ ]:
G4 = nx.DiGraph()
add_edges_with_weights2(G4, seq_copy)
#G4.nodes()
r = create_folium_map2(G4)
#r.save('osszevont.html')
r

###Mav modes combining + heatmap

In [ ]:
stops_mav = stops_mav.drop(columns = ['stop_code','stop_desc','wheelchair_boarding','stop_timezone'])
for _, row in stops_mav.iterrows():
    stop_name = row['stop_name']
    stop_id = row['stop_id']
    coordinate = (row['stop_lon'], row['stop_lat'])
    stops_list.append([stop_name, stop_id, coordinate])
for stop in stops_list:
    stop[0] = remove_brackets_and_extras(stop[0])
stops_dict = defaultdict(list)
for stop_name, stop_id, coord in stops_list:
    stops_dict[stop_name].append([stop_id, coord])
grouped_by_stop = []
for stop_name, entries in stops_dict.items():
    groups = group_coordinates_by_distance(entries)
    grouped_by_stop.extend(groups)
calculate_average_coordinates(grouped_by_stop)
average_coords_dict = {}
for group in grouped_by_stop:
    for stop_id, avg_coord in group:
        average_coords_dict[stop_id] = avg_coord
seq_copy2 = copy.deepcopy(segments_freq_mav)
seq_copy2['geometry'] = seq_copy2.apply(update_geometry, axis=1)

In [ ]:
seq_copy2['weight'] = seq_copy2['ntrips'] * seq_copy2['capacity']
seq_copy2['normalized_weight'] = seq_copy2['weight'] / seq_copy2['weight'].max()
mean_x, mean_y = seq_copy2.geometry.centroid.x.mean(), seq_copy2.geometry.centroid.y.mean()
m = folium.Map(location=[mean_y, mean_x], zoom_start=12)
colormap = cmb.LinearColormap(colors=['blue', 'green', 'yellow', 'red'], vmin=0, vmax=1)
for _, row in seq_copy2.iterrows():
    weight = row['normalized_weight']
    locations = [(coord[1], coord[0]) for coord in row['geometry'].coords]
    folium.PolyLine(
        locations=locations,
        weight=weight * 10 + 0.1,  # Adjust thickness
        color=colormap(weight),  # Color based on weight
        opacity=0.7
    ).add_to(m)
colormap.caption = 'Heatmap Intensity'
m.add_child(colormap)
m

###Zones and OD matrix

In [ ]:
sf = shp.Reader("zone_zone.shp")
#print(type(sf))

In [ ]:
def read_shapefile_df(sf):
    """
    Read a shapefile into a Pandas dataframe with a 'coords'
    column holding the geometry information. This uses the pyshp
    package
    """
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    shps = [s.points for s in sf.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df

In [ ]:
def read_shapefile_gdf(sf):
    """
    Read a shapefile into a GeoDataFrame.
    """
    # Extract the fields and records
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()

    # Extract the shapes and convert them to shapely geometries
    geometries = []
    for shape in sf.shapes():
        if len(shape.points) == 1:  # Point
            geometries.append(Point(shape.points[0]))
        else:  # Polygon
            geometries.append(Polygon(shape.points))

    # Create a GeoDataFrame
    df = gpd.GeoDataFrame(columns=fields, data=records)
    df = df.assign(geometry=geometries)

    return df

In [ ]:
df_shape = read_shapefile_gdf(sf)
df_shape

In [ ]:
def plot_map(sf, df, x_lim=None, y_lim=None, figsize=(33, 25), filtered_ids=None):
    '''
    Plot map with lim coordinates and colored shapes based on 'NO' attribute.
    '''
    plt.figure(figsize=figsize)
    id = 0

    cmap = plt.get_cmap('viridis')
    norm = plt.Normalize(df['NO'].min(), df['NO'].max())

    for shape in sf.shapeRecords():
        if filtered_ids is not None and id not in filtered_ids:
            id += 1
            continue
        x = [i[0] for i in shape.shape.points[:]]
        y = [i[1] for i in shape.shape.points[:]]
        color = cmap(norm(df.loc[id, 'NO']))
        plt.fill(x, y, color=color, edgecolor='k')

        if x_lim is None and y_lim is None:
            x0 = np.mean(x)
            y0 = np.mean(y)
            plt.text(x0, y0, id, fontsize=10)
        id += 1

    if x_lim is not None and y_lim is not None:
        plt.xlim(x_lim)
        plt.ylim(y_lim)

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    plt.colorbar(sm, orientation='vertical', label='NO Attribute')
    #plt.savefig('zonak.png')
    plt.show()


In [ ]:
# Filter dataframe based on 'NO' attribute
filtered_df = df_shape[(df_shape['NO'] >= 1000) & (df_shape['NO'] <= 23999)]
filtered_ids = filtered_df.index.tolist()

#print(filtered_df.shape)  # Print the shape of the filtered dataframe

# Plot the filtered map with colors
plot_map(sf, filtered_df, filtered_ids=filtered_ids)

In [ ]:
geom = copy.deepcopy(seq_copy)
geom.head()

In [ ]:
geom2 = copy.deepcopy(segments_freq)
geom2.head()

In [ ]:
if geom.crs is None:
    # Replace 'EPSG:4326' with the correct CRS if known
    geom.set_crs(epsg=4326, inplace=True)

# Define the path to the shapefile
shapefile_path = 'zone_zone.SHP'

# Check if the shapefile exists
if not os.path.exists(shapefile_path):
    raise FileNotFoundError(f"Shapefile not found: {shapefile_path}")

# Load the shapefile using Fiona
with fiona.open(shapefile_path) as shapefile:
    # Ensure shapefile has a CRS
    if shapefile.crs is None:
        # Replace 'EPSG:4326' with the correct CRS if known
        shapefile_crs = 'EPSG:4326'
    else:
        shapefile_crs = shapefile.crs

    # Convert the shapefile features to GeoDataFrame
    shapefile_gdf = gpd.GeoDataFrame.from_features(shapefile, crs=shapefile_crs)

    # Filter the shapefile GeoDataFrame based on the 'NO' attribute
    shapefile_filtered = shapefile_gdf[(shapefile_gdf['NO'] >= 1000) & (shapefile_gdf['NO'] <= 23999)]

    # Ensure CRS match
    if shapefile_filtered.crs != geom.crs:
        shapefile_filtered = shapefile_filtered.to_crs(geom.crs)

    # Initialize the new columns with None values
    geom['first_coord_shape_index'] = None
    geom['second_coord_shape_index'] = None

    # Convert filtered shapes to Shapely geometries
    filtered_shapes = [(shape(geom_shape), shape_id) for geom_shape, shape_id in zip(shapefile_filtered.geometry, shapefile_filtered.index)]

    # Iterate through the rows of the GeoDataFrame
    for idx, row in geom.iterrows():
        # Extract the coordinates from the geometry column
        coords = list(row.geometry.coords)
        first_coord = Point(coords[0])
        second_coord = Point(coords[-1])
        """if idx <= 10:
          print(coords)
          print(first_coord)
          print(second_coord)"""

        # Check which shape the first coordinate falls within
        first_match = next((shape_id for geom_shape, shape_id in filtered_shapes if geom_shape.contains(first_coord)), None)
        geom.at[idx, 'first_coord_shape_index'] = first_match

        # Check which shape the second coordinate falls within
        second_match = next((shape_id for geom_shape, shape_id in filtered_shapes if geom_shape.contains(second_coord)), None)
        geom.at[idx, 'second_coord_shape_index'] = second_match

In [ ]:
if geom2.crs is None:
    # Replace 'EPSG:4326' with the correct CRS if known
    geom2.set_crs(epsg=4326, inplace=True)

# Define the path to the shapefile
shapefile_path = 'zone_zone.SHP'

# Check if the shapefile exists
if not os.path.exists(shapefile_path):
    raise FileNotFoundError(f"Shapefile not found: {shapefile_path}")

# Load the shapefile using Fiona
with fiona.open(shapefile_path) as shapefile:
    # Ensure shapefile has a CRS
    if shapefile.crs is None:
        # Replace 'EPSG:4326' with the correct CRS if known
        shapefile_crs = 'EPSG:4326'
    else:
        shapefile_crs = shapefile.crs

    # Convert the shapefile features to GeoDataFrame
    shapefile_gdf = gpd.GeoDataFrame.from_features(shapefile, crs=shapefile_crs)

    # Filter the shapefile GeoDataFrame based on the 'NO' attribute
    shapefile_filtered = shapefile_gdf[(shapefile_gdf['NO'] >= 1000) & (shapefile_gdf['NO'] <= 23999)]

    # Ensure CRS match
    if shapefile_filtered.crs != geom2.crs:
        shapefile_filtered = shapefile_filtered.to_crs(geom2.crs)

    # Initialize the new columns with None values
    geom2['first_coord_shape_index'] = None
    geom2['second_coord_shape_index'] = None

    # Convert filtered shapes to Shapely geometries
    filtered_shapes = [(shape(geom_shape), shape_id) for geom_shape, shape_id in zip(shapefile_filtered.geometry, shapefile_filtered.index)]

    # Iterate through the rows of the GeoDataFrame
    for idx, row in geom2.iterrows():
        # Extract the coordinates from the geometry column
        coords = list(row.geometry.coords)
        first_coord = Point(coords[0])
        second_coord = Point(coords[-1])
        """if idx <= 10:
          print(coords)
          print(first_coord)
          print(second_coord)"""

        # Check which shape the first coordinate falls within
        first_match = next((shape_id for geom_shape, shape_id in filtered_shapes if geom_shape.contains(first_coord)), None)
        geom2.at[idx, 'first_coord_shape_index'] = first_match

        # Check which shape the second coordinate falls within
        second_match = next((shape_id for geom_shape, shape_id in filtered_shapes if geom_shape.contains(second_coord)), None)
        geom2.at[idx, 'second_coord_shape_index'] = second_match

In [ ]:
# Initialize an empty dictionary to store the results.
node_shape_dict = {}

# Iterate over each row in the GeoDataFrame.
for index, row in geom.iterrows():
    # Extract the geometry (assuming it's a LineString).
    line = row['geometry']

    # Get the first and last elements of the LineString.
    first_point = line.coords[0]
    last_point = line.coords[-1]

    # Swap the longitude and latitude
    first_point_swapped = (first_point[1], first_point[0])
    last_point_swapped = (last_point[1], last_point[0])

    # Check if the first point is not already in the dictionary.
    if first_point_swapped not in node_shape_dict:
        # Add the swapped first point as a key with the first_shape_index as the value.
        node_shape_dict[first_point_swapped] = row['first_coord_shape_index']

    # Check if the last point is not already in the dictionary.
    if last_point_swapped not in node_shape_dict:
        # Add the swapped last point as a key with the second_shape_index as the value.
        node_shape_dict[last_point_swapped] = row['second_coord_shape_index']

In [ ]:
#print(len(node_shape_dict))
#print(list(node_shape_dict.items())[23])
print(node_shape_dict)

In [ ]:
# Initialize the matrix
matrix_size = len(shapefile_filtered)
matrix_cap = np.zeros((matrix_size, matrix_size))

# Populate the matrix based on geom rows
for idx, row in geom.iterrows():
    first_index = row['first_coord_shape_index']
    second_index = row['second_coord_shape_index']
    if first_index is not None and second_index is not None and first_index <= 921 and second_index <= 921:
      matrix_cap[first_index, second_index] += row['ntrips'] * row['capacity']
matrix_trip = np.zeros((matrix_size, matrix_size))
for idx, row in geom.iterrows():
    first_index = row['first_coord_shape_index']
    second_index = row['second_coord_shape_index']
    if first_index is not None and second_index is not None and first_index <= 921 and second_index <= 921:
      matrix_trip[first_index, second_index] += row['ntrips']
matrix_cap

###Be- és kimenő súlyok, OD összehasonlítás

In [ ]:
file_path = 'OD_tk.xlsx'
df = pd.read_excel(file_path)
third_column_tk = df.iloc[2:924, 2].reset_index(drop=True)
third_row_tk = df.iloc[1, 3:925].reset_index(drop=True)

file_path2 = 'OD_szgk.xlsx'
df2 = pd.read_excel(file_path2)
third_column_szgk = df2.iloc[2:924, 2].reset_index(drop=True)
third_row_szgk = df2.iloc[1, 3:925].reset_index(drop=True)

third_column_tk = third_column_tk / 10
third_row_tk = third_row_tk / 10
third_column_szgk = third_column_szgk / 10 *1.5
third_row_szgk = third_row_szgk / 10 *1.5

In [ ]:
row_sums_without_diagonal = []
for i in range(matrix_cap.shape[0]):
    row_sum = np.sum(matrix_cap[i]) - matrix_cap[i, i]
    row_sums_without_diagonal.append(row_sum)

ki_tk = pd.DataFrame({'index': third_column_tk.index, 'OD_outvalue': third_column_tk, 'calculated_outvalue': row_sums_without_diagonal})

col_sums_without_diagonal = []
for i in range(matrix_cap.shape[1]):
    col_sum = np.sum(matrix_cap[:, i]) - matrix_cap[i, i]
    col_sums_without_diagonal.append(col_sum)

# 3. Append the computed sums as a new column to the `third_column` DataFrame
#third_row['calculated_invalue'] = col_sums_without_diagonal
be_tk = pd.DataFrame({'index': third_row_tk.index, 'OD_invalue': third_row_tk, 'calculated_invalue': col_sums_without_diagonal})

ki_tk.set_index('index', inplace=True)
be_tk.set_index('index', inplace=True)

be_tk.head(2)

In [ ]:
row_sums_without_diagonal2 = []
for i in range(matrix_cap.shape[0]):
    row_sum = np.sum(matrix_cap[i]) - matrix_cap[i, i]
    row_sums_without_diagonal2.append(row_sum)

ki_szgk = pd.DataFrame({'index': third_column_szgk.index, 'OD_outvalue': third_column_szgk, 'calculated_outvalue': row_sums_without_diagonal})

col_sums_without_diagonal2 = []
for i in range(matrix_cap.shape[1]):
    col_sum = np.sum(matrix_cap[:, i]) - matrix_cap[i, i]
    col_sums_without_diagonal2.append(col_sum)

# 3. Append the computed sums as a new column to the `third_column` DataFrame
#third_row['calculated_invalue'] = col_sums_without_diagonal
be_szgk = pd.DataFrame({'index': third_row_szgk.index, 'OD_invalue': third_row_szgk, 'calculated_invalue': col_sums_without_diagonal})

ki_szgk.set_index('index', inplace=True)
be_szgk.set_index('index', inplace=True)

be_szgk.head(2)

In [ ]:
ki_tk['OD_outvalue'] = pd.to_numeric(ki_tk['OD_outvalue'], errors='coerce')
ki_tk['calculated_outvalue'] = pd.to_numeric(ki_tk['calculated_outvalue'], errors='coerce')
ki_tk['Out_difference'] = - ki_tk['OD_outvalue'] + ki_tk['calculated_outvalue']
be_tk['OD_invalue'] = pd.to_numeric(be_tk['OD_invalue'], errors='coerce')
be_tk['calculated_invalue'] = pd.to_numeric(be_tk['calculated_invalue'], errors='coerce')
be_tk['In_difference'] = -be_tk['OD_invalue'] + be_tk['calculated_invalue']

ki_szgk['OD_outvalue'] = pd.to_numeric(ki_szgk['OD_outvalue'], errors='coerce')
ki_szgk['calculated_outvalue'] = pd.to_numeric(ki_szgk['calculated_outvalue'], errors='coerce')
ki_szgk['Out_difference'] = - ki_szgk['OD_outvalue'] + ki_szgk['calculated_outvalue']
be_szgk['OD_invalue'] = pd.to_numeric(be_szgk['OD_invalue'], errors='coerce')
be_szgk['calculated_invalue'] = pd.to_numeric(be_szgk['calculated_invalue'], errors='coerce')
be_szgk['In_difference'] = -be_szgk['OD_invalue'] + be_szgk['calculated_invalue']

In [ ]:
min_index_ki_szgk = ki_szgk['Out_difference'].idxmin()  # Get index of the min value
max_index_ki_szgk = ki_szgk['Out_difference'].idxmax()  # Get index of the max value
min_index_be_szgk = be_szgk['In_difference'].idxmin()
max_index_be_szgk = be_szgk['In_difference'].idxmax()

min_ki_szgk = ki_szgk['Out_difference'].min()  # Get the min value
max_ki_szgk = ki_szgk['Out_difference'].max()  # Get the max value
min_be_szgk = be_szgk['In_difference'].min()
max_be_szgk = be_szgk['In_difference'].max()

min_index_ki_tk = ki_tk['Out_difference'].idxmin()  # Get index of the min value
max_index_ki_tk = ki_tk['Out_difference'].idxmax()  # Get index of the max value
min_index_be_tk = be_tk['In_difference'].idxmin()
max_index_be_tk = be_tk['In_difference'].idxmax()

min_ki_tk = ki_tk['Out_difference'].min()  # Get the min value
max_ki_tk = ki_tk['Out_difference'].max()  # Get the max value
min_be_tk = be_tk['In_difference'].min()
max_be_tk = be_tk['In_difference'].max()

###Difference maps

In [ ]:
gdf = gpd.read_file('zone_zone.SHP')
gdf = gdf[(gdf['NO'] >= 1000) & (gdf['NO'] <= 23999)]
# Convert the GeoDataFrame to the same CRS as Folium (WGS84 - EPSG:4326)
gdf = gdf.to_crs(epsg=4326)

# Create a base map
m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=10)

# Function to style the shapes
def style_function(feature):
    index = feature['properties']['idx']  # Use 'idx' which we'll add to properties
    if index == min_index_ki_tk:
        return {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.7}
    elif index == max_index_ki_tk:
        return {'fillColor': 'red', 'color': 'red', 'weight': 2, 'fillOpacity': 0.7}
    else:
        return {'fillColor': 'transparent', 'color': 'black', 'weight': 1, 'fillOpacity': 0.0}

# Add each shape to the map with its index in properties
for idx, row in gdf.iterrows():
    # Add the appropriate tooltip based on whether it's min, max, or neither
    if idx == min_index_ki_tk:
        tooltip = f"Min Value: {min_ki_tk}"
    elif idx == max_index_ki_tk:
        tooltip = f"Max Value: {max_ki_tk}"
    else:
        tooltip = None  # No tooltip for other shapes

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}  # Add the index to properties
        },
        style_function=style_function,
        tooltip=tooltip  # Attach the tooltip with min/max value
    )
    geojson.add_to(m)

# Add an explanation for the colors on the map
folium.Marker(
    location=[gdf.geometry.centroid.y.mean() + 0.11, gdf.geometry.centroid.x.mean() + 0.1],
    icon=DivIcon(
        icon_size=(80, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 8pt; color: red;">&#9679; Max amount of out capacity</div>',
    )
).add_to(m)

folium.Marker(
    location=[gdf.geometry.centroid.y.mean() + 0.08, gdf.geometry.centroid.x.mean() + 0.1],
    icon=DivIcon(
        icon_size=(80, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 8pt; color: blue;">&#9679; Min amount of out capacity</div>',
    )
).add_to(m)

# Display the map
#m.save('minmax_ki.html')
m

In [ ]:
# Convert the GeoDataFrame to the same CRS as Folium (WGS84 - EPSG:4326)
#gdf = gdf.to_crs(epsg=4326)

# Create a base map
m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=10)

# Function to style the shapes
def style_function(feature):
    index = feature['properties']['idx']  # Use 'idx' which we'll add to properties
    if index == min_index_be_tk:
        return {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.7}
    elif index == max_index_be_tk:
        return {'fillColor': 'red', 'color': 'red', 'weight': 2, 'fillOpacity': 0.7}
    else:
        return {'fillColor': 'transparent', 'color': 'black', 'weight': 1, 'fillOpacity': 0.0}

# Add each shape to the map with its index in properties
for idx, row in gdf.iterrows():
    # Add the appropriate tooltip based on whether it's min, max, or neither
    if idx == min_index_be_tk:
        tooltip = f"Min Value: {min_be_tk}"
    elif idx == max_index_be_tk:
        tooltip = f"Max Value: {max_be_tk}"
    else:
        tooltip = None  # No tooltip for other shapes

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}  # Add the index to properties
        },
        style_function=style_function,
        tooltip=tooltip  # Attach the tooltip with min/max value
    )
    geojson.add_to(m)

# Add an explanation for the colors on the map
folium.Marker(
    location=[gdf.geometry.centroid.y.mean() + 0.11, gdf.geometry.centroid.x.mean() + 0.1],
    icon=DivIcon(
        icon_size=(80, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 8pt; color: red;">&#9679; Max amount of in capacity</div>',
    )
).add_to(m)

folium.Marker(
    location=[gdf.geometry.centroid.y.mean() + 0.08, gdf.geometry.centroid.x.mean() + 0.1],
    icon=DivIcon(
        icon_size=(80, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 8pt; color: blue;">&#9679; Min amount of in capacity</div>',
    )
).add_to(m)

# Display the map
#m.save('minmax_be.html')
m

In [ ]:
# Normalize the values so that 0 maps to white, negatives to blue, and positives to red
norm = mcolors.TwoSlopeNorm(vmin=min_ki_tk, vcenter=0, vmax=max_ki_tk)

# Use a colormap that transitions from blue (negative) to white (neutral) to red (positive)
cmap = mcolors.LinearSegmentedColormap.from_list("pos_neg_cmap", ['blue', 'white', 'red'])

# Convert the filtered GeoDataFrame to the same CRS as Folium (WGS84 - EPSG:4326)
gdf_filtered = gdf.to_crs(epsg=4326)

# Function to get the color based on the normalized value
def get_color(value):
    norm_value = norm(value)  # Normalize the value with center at 0
    return mcolors.to_hex(cmap(norm_value))  # Convert color to hex for folium

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the value
for idx, row in gdf_filtered.iterrows():
    value = ki_tk.iloc[idx, 2]  # Get the corresponding value from the DataFrame
    color = get_color(value)  # Get the color based on the value

    tooltip = f"Value: {value}"  # Tooltip to display the value on hover

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value
    )
    geojson.add_to(m)

# Create a colormap legend using branca
colormap = cmb.LinearColormap(
    colors=['blue', 'white', 'red'],
    vmin=min_ki_tk, vmax=max_ki_tk,
    caption='Out Value'
)
colormap.add_to(m)

# Save the map to an HTML file and display it
#m.save('map.html')
m

In [ ]:
# Normalize the values so that 0 maps to white, negatives to blue, and positives to red
norm = mcolors.TwoSlopeNorm(vmin=min_be_tk, vcenter=0, vmax=max_be_tk)

# Use a colormap that transitions from blue (negative) to white (neutral) to red (positive)
cmap = mcolors.LinearSegmentedColormap.from_list("pos_neg_cmap", ['blue', 'white', 'red'])

# Convert the filtered GeoDataFrame to the same CRS as Folium (WGS84 - EPSG:4326)
gdf_filtered = gdf.to_crs(epsg=4326)

# Function to get the color based on the normalized value
def get_color(value):
    norm_value = norm(value)  # Normalize the value with center at 0
    return mcolors.to_hex(cmap(norm_value))  # Convert color to hex for folium

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the value
for idx, row in gdf_filtered.iterrows():
    value = ki_tk.iloc[idx, 2]  # Get the corresponding value from the DataFrame
    color = get_color(value)  # Get the color based on the value

    tooltip = f"Value: {value}"  # Tooltip to display the value on hover

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value
    )
    geojson.add_to(m)

# Create a colormap legend using branca
colormap = cmb.LinearColormap(
    colors=['blue', 'white', 'red'],
    vmin=min_be_tk, vmax=max_be_tk,
    caption='In Value'
)
colormap.add_to(m)

# Save the map to an HTML file and display it
#m.save('map.html')
m

In [ ]:
# Normalize the values to the range [0, 1] for color mapping
norm = plt.Normalize(vmin=min_ki_tk, vmax=max_ki_tk)

# Use the 'viridis' colormap for a balanced color gradient
cmap = mcolors.LinearSegmentedColormap.from_list("weight_cmap", ['blue','green','yellow', 'orange', 'red'])

# Convert the filtered GeoDataFrame to the same CRS as Folium (WGS84 - EPSG:4326)
gdf_filtered = gdf.to_crs(epsg=4326)

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the value
for idx, row in gdf_filtered.iterrows():
    value = ki_tk.iloc[idx, 2]  # Get the corresponding value from the DataFrame
    color = get_color(value)  # Get the color based on the value

    tooltip = f"Value: {value}"  # Tooltip to display the value on hover

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value
    )
    geojson.add_to(m)

# Create a colormap legend using branca
colormap = cmb.LinearColormap(
    colors=['blue','green','yellow', 'orange', 'red'],
    vmin=min_ki_tk, vmax=max_ki_tk,
    caption='Out Value'
)
colormap.add_to(m)

# Save the map to an HTML file and display it
#m.save('percent_ki.html')
m

In [ ]:
# Normalize the values to the range [0, 1] for color mapping
norm = plt.Normalize(vmin=min_be_tk, vmax=max_be_tk)

# Use the 'viridis' colormap for a balanced color gradient
cmap = mcolors.LinearSegmentedColormap.from_list("weight_cmap", ['blue','green','yellow', 'orange', 'red'])

# Convert the filtered GeoDataFrame to the same CRS as Folium (WGS84 - EPSG:4326)
gdf_filtered = gdf.to_crs(epsg=4326)

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the value
for idx, row in gdf_filtered.iterrows():
    value = be_tk.iloc[idx, 2]  # Get the corresponding value from the DataFrame
    color = get_color(value)  # Get the color based on the value

    tooltip = f"Value: {value}"  # Tooltip to display the value on hover

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value
    )
    geojson.add_to(m)

# Create a colormap legend using branca
colormap = cmb.LinearColormap(
    colors=['blue','green','yellow', 'orange', 'red'],
    vmin=min_be_tk, vmax=max_be_tk,
    caption='In Value'
)
colormap.add_to(m)

# Save the map to an HTML file and display it
#m.save('percent_be.html')
m

In [ ]:
# Assume be.iloc[:, 2] contains the values you want to visualize
values = ki_tk.iloc[:, 2]

# Calculate percentiles
percentiles = np.percentile(values, np.arange(0, 101, 10))  # Dividing into 10 percentiles (0-10, 10-20, ..., 90-100)

# Create a blue-to-red colormap
cmap = mcolors.LinearSegmentedColormap.from_list("blue_red_cmap", ["blue","white", "red"])

# Function to assign colors based on the percentile range
def get_percentile_color(value):
    for i in range(len(percentiles) - 1):
        if percentiles[i] <= value < percentiles[i + 1]:
            color = cmap(i / (len(percentiles) - 1))  # Use blue-to-red colormap
            return mcolors.to_hex(color)
    return mcolors.to_hex(cmap(1.0))  # For the top 100th percentile

# Convert the filtered GeoDataFrame to the same CRS as Folium (WGS84 - EPSG:4326)
gdf_filtered = gdf.to_crs(epsg=4326)

# Add percentiles as a new column to the GeoDataFrame for convenience (optional)
gdf_filtered['percentile'] = np.searchsorted(percentiles, values, side='right')

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the percentile
for idx, row in gdf_filtered.iterrows():
    value = values[idx]
    color = get_percentile_color(value)  # Get the color based on the percentile

    tooltip = f"Value: {value}, Percentile: {gdf_filtered['percentile'][idx]}"  # Tooltip with value and percentile

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value and percentile
    )
    geojson.add_to(m)

# Create a colormap legend using branca
colormap = cmb.LinearColormap(
    colors=['blue', 'white', 'red'],
    vmin=percentiles[0], vmax=percentiles[-1],
    caption='Out Percentile Range'
)
colormap.add_to(m)  # Add the colormap to the map

# Display the map
#m.save('percent_ki.html')
m

In [ ]:
values = be_tk.iloc[:, 2]

# Calculate percentiles
percentiles = np.percentile(values, np.arange(0, 101, 10))  # Dividing into 10 percentiles (0-10, 10-20, ..., 90-100)

# Create a blue-to-red colormap
cmap = mcolors.LinearSegmentedColormap.from_list("blue_red_cmap", ["blue","white", "red"])

# Function to assign colors based on the percentile range
def get_percentile_color(value):
    for i in range(len(percentiles) - 1):
        if percentiles[i] <= value < percentiles[i + 1]:
            color = cmap(i / (len(percentiles) - 1))  # Use blue-to-red colormap
            return mcolors.to_hex(color)
    return mcolors.to_hex(cmap(1.0))  # For the top 100th percentile

# Convert the filtered GeoDataFrame to the same CRS as Folium (WGS84 - EPSG:4326)
gdf_filtered = gdf.to_crs(epsg=4326)

# Add percentiles as a new column to the GeoDataFrame for convenience (optional)
gdf_filtered['percentile'] = np.searchsorted(percentiles, values, side='right')

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the percentile
for idx, row in gdf_filtered.iterrows():
    value = values[idx]
    color = get_percentile_color(value)  # Get the color based on the percentile

    tooltip = f"Value: {value}, Percentile: {gdf_filtered['percentile'][idx]}"  # Tooltip with value and percentile

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value and percentile
    )
    geojson.add_to(m)

# Create a colormap legend using branca
colormap = cmb.LinearColormap(
    colors=['blue', 'white', 'red'],
    vmin=percentiles[0], vmax=percentiles[-1],
    caption='In Percentile Range'
)
colormap.add_to(m)  # Add the colormap to the map

# Display the map
#m.save('percent_be.html')
m

In [ ]:
values = ki_tk.iloc[:, 2]

# Reset index of gdf_filtered to ensure it matches with the values
#gdf_filtered = gdf_filtered.iloc[:-1].reset_index(drop=True)  # Also drop the last row in gdf_filtered

# Define color ranges and corresponding intervals
def get_custom_color(idx, value):
    # Check if the shape would be colored purple or blue and if be_tk.iloc[idx, 1] is 0
    if value < -450:
        if ki_tk.iloc[idx, 1] == 0:
            return '#000000'  # Black if be_tk.iloc[idx, 1] is 0
        return '#800080'  # Purple for values < -450
    elif -450 <= value < 0:
        if ki_tk.iloc[idx, 1] == 0:
            return '#000000'  # Black if be_tk.iloc[idx, 1] is 0
        return '#0000FF'  # Blue for values between -450 and 0
    elif value == 0:
        return '#FFFF00'  # Yellow for values equal to 0
    elif 0 < value <= 450:
        return '#FFA500'  # Orange for values between 0 and 450
    elif value > 450:
        return '#FF0000'  # Red for values greater than 450
    return '#FFFFFF'  # Default to white if something goes wrong

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the value interval
for idx, row in gdf_filtered.iterrows():
    value = values.iloc[idx]  # Get the value
    color = get_custom_color(idx, value)  # Get the color based on the defined intervals and conditions

    tooltip = f"Value: {value}"  # Tooltip with value

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value
    )
    geojson.add_to(m)

# Display the map
#m.save('szines_elotte_ki.html')
m

In [ ]:
values = be_tk.iloc[:, 2]

# Reset index of gdf_filtered to ensure it matches with the values
#gdf_filtered = gdf_filtered.iloc[:-1].reset_index(drop=True)  # Also drop the last row in gdf_filtered

# Define color ranges and corresponding intervals
def get_custom_color(idx, value):
    # Check if the shape would be colored purple or blue and if be_tk.iloc[idx, 1] is 0
    if value < -450:
        if be_tk.iloc[idx, 1] == 0:
            return '#000000'  # Black if be_tk.iloc[idx, 1] is 0
        return '#800080'  # Purple for values < -450
    elif -450 <= value < 0:
        if be_tk.iloc[idx, 1] == 0:
            return '#000000'  # Black if be_tk.iloc[idx, 1] is 0
        return '#0000FF'  # Blue for values between -450 and 0
    elif value == 0:
        return '#FFFF00'  # Yellow for values equal to 0
    elif 0 < value <= 450:
        return '#FFA500'  # Orange for values between 0 and 450
    elif value > 450:
        return '#FF0000'  # Red for values greater than 450
    return '#FFFFFF'  # Default to white if something goes wrong

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the value interval
for idx, row in gdf_filtered.iterrows():
    value = values.iloc[idx]  # Get the value
    color = get_custom_color(idx, value)  # Get the color based on the defined intervals and conditions

    tooltip = f"Value: {value}"  # Tooltip with value

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value
    )
    geojson.add_to(m)

# Display the map
#m.save('szines_elotte_be.html')
m

In [ ]:
values = ki_tk.iloc[:, 1] - ki_tk.iloc[:, 0] - ki_szgk.iloc[:, 0]

# Reset index of gdf_filtered to ensure it matches with the values
#gdf_filtered = gdf_filtered.iloc[:-1].reset_index(drop=True)  # Also drop the last row in gdf_filtered

# Define color ranges and corresponding intervals
def get_custom_color(idx, value):
    # Check if the shape would be colored purple or blue and if be_tk.iloc[idx, 1] is 0
    if value < -450:
        if ki_tk.iloc[idx, 1] == 0:
            return '#000000'  # Black if be_tk.iloc[idx, 1] is 0
        return '#800080'  # Purple for values < -450
    elif -450 <= value < 0:
        if ki_tk.iloc[idx, 1] == 0:
            return '#000000'  # Black if be_tk.iloc[idx, 1] is 0
        return '#0000FF'  # Blue for values between -450 and 0
    elif value == 0:
        return '#FFFF00'  # Yellow for values equal to 0
    elif 0 < value <= 450:
        return '#FFA500'  # Orange for values between 0 and 450
    elif value > 450:
        return '#FF0000'  # Red for values greater than 450
    return '#FFFFFF'  # Default to white if something goes wrong

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the value interval
for idx, row in gdf_filtered.iterrows():
    value = values.iloc[idx]  # Get the value
    color = get_custom_color(idx, value)  # Get the color based on the defined intervals and conditions

    tooltip = f"Value: {value}"  # Tooltip with value

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value
    )
    geojson.add_to(m)

# Display the map
m.save('szines_utana_ki.html')
m

In [ ]:
values = be_tk.iloc[:, 1] - be_tk.iloc[:, 0] - be_szgk.iloc[:, 0]

# Reset index of gdf_filtered to ensure it matches with the values
#gdf_filtered = gdf_filtered.iloc[:-1].reset_index(drop=True)  # Also drop the last row in gdf_filtered

# Define color ranges and corresponding intervals
def get_custom_color(idx, value):
    # Check if the shape would be colored purple or blue and if be_tk.iloc[idx, 1] is 0
    if value < -450:
        if be_tk.iloc[idx, 1] == 0:
            return '#000000'  # Black if be_tk.iloc[idx, 1] is 0
        return '#800080'  # Purple for values < -450
    elif -450 <= value < 0:
        if be_tk.iloc[idx, 1] == 0:
            return '#000000'  # Black if be_tk.iloc[idx, 1] is 0
        return '#0000FF'  # Blue for values between -450 and 0
    elif value == 0:
        return '#FFFF00'  # Yellow for values equal to 0
    elif 0 < value <= 450:
        return '#FFA500'  # Orange for values between 0 and 450
    elif value > 450:
        return '#FF0000'  # Red for values greater than 450
    return '#FFFFFF'  # Default to white if something goes wrong

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the value interval
for idx, row in gdf_filtered.iterrows():
    value = values.iloc[idx]  # Get the value
    color = get_custom_color(idx, value)  # Get the color based on the defined intervals and conditions

    tooltip = f"Value: {value}"  # Tooltip with value

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value
    )
    geojson.add_to(m)

# Display the map
m.save('szines_utana_be.html')
m

In [ ]:
values  = ki_szgk.iloc[:, 0]

# Reset index of gdf_filtered to ensure it matches with the values
#gdf_filtered = gdf_filtered.iloc[:-1].reset_index(drop=True)  # Also drop the last row in gdf_filtered

# Define color ranges and corresponding intervals
def get_custom_color(value):
    if value < -450:
        return '#800080'  # Purple
    elif -450 <= value < 0:
        return '#0000FF'  # Blue
    elif value == 0:
        return '#FFFF00'  # Yellow
    elif 0 < value <= 450:
        return '#FFA500'  # Orange
    elif value > 450:
        return '#FF0000'  # Red
    return '#FFFFFF'  # Default to white if something goes wrong

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the value interval
for idx, row in gdf_filtered.iterrows():
    value = values.iloc[idx]  # Get the value
    color = get_custom_color(value)  # Get the color based on the defined intervals

    tooltip = f"Value: {value}"  # Tooltip with value

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value
    )
    geojson.add_to(m)

# Display the map
#m.save('szgk_ki.html')
m

In [ ]:
values  = be_szgk.iloc[:, 0]

# Reset index of gdf_filtered to ensure it matches with the values
#gdf_filtered = gdf_filtered.iloc[:-1].reset_index(drop=True)  # Also drop the last row in gdf_filtered

# Define color ranges and corresponding intervals
def get_custom_color(value):
    if value < -450:
        return '#800080'  # Purple
    elif -450 <= value < 0:
        return '#0000FF'  # Blue
    elif value == 0:
        return '#FFFF00'  # Yellow
    elif 0 < value <= 450:
        return '#FFA500'  # Orange
    elif value > 450:
        return '#FF0000'  # Red
    return '#FFFFFF'  # Default to white if something goes wrong

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the value interval
for idx, row in gdf_filtered.iterrows():
    value = values.iloc[idx]  # Get the value
    color = get_custom_color(value)  # Get the color based on the defined intervals

    tooltip = f"Value: {value}"  # Tooltip with value

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value
    )
    geojson.add_to(m)

# Display the map
#m.save('szgk_be.html')
m

### Fraction maps

In [ ]:
ki_tk['Out_fraction'] =  ki_tk['OD_outvalue']/ ki_tk['calculated_outvalue']
be_tk['In_fraction'] =  be_tk['OD_invalue']/ be_tk['calculated_invalue']

ki_szgk['Out_fraction'] =  ki_szgk['OD_outvalue'] / ki_szgk['calculated_outvalue']
be_szgk['In_fraction'] =  be_szgk['OD_invalue'] / be_szgk['calculated_invalue']

In [ ]:
min_index_ki_tk = ki_tk['Out_fraction'].idxmin()  # Get index of the min value
max_index_ki_tk = ki_tk['Out_fraction'].idxmax()  # Get index of the max value
min_index_be_tk = be_tk['In_fraction'].idxmin()
max_index_be_tk = be_tk['In_fraction'].idxmax()

min_ki_tk = ki_tk['Out_fraction'].min()  # Get the min value
max_ki_tk = ki_tk['Out_fraction'].max()  # Get the max value
min_be_tk = be_tk['In_fraction'].min()
max_be_tk = be_tk['In_fraction'].max()

min_index_ki_szgk = ki_szgk['Out_fraction'].idxmin()  # Get index of the min value
max_index_ki_szgk = ki_szgk['Out_fraction'].idxmax()  # Get index of the max value
min_index_be_szgk = be_szgk['In_fraction'].idxmin()
max_index_be_szgk = be_szgk['In_fraction'].idxmax()

min_ki_szgk = ki_szgk['Out_fraction'].min()  # Get the min value
max_ki_szgk = ki_szgk['Out_fraction'].max()  # Get the max value
min_be_szgk = be_szgk['In_fraction'].min()
max_be_szgk = be_szgk['In_fraction'].max()

In [ ]:
ki_tk.head()

In [ ]:
# Create a base map
m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=10)

# Function to style the shapes
def style_function(feature):
    index = feature['properties']['idx']  # Use 'idx' which we'll add to properties
    if index == min_index_ki_tk:
        return {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.7}
    elif index == max_index_ki_tk:
        return {'fillColor': 'red', 'color': 'red', 'weight': 2, 'fillOpacity': 0.7}
    else:
        return {'fillColor': 'transparent', 'color': 'black', 'weight': 1, 'fillOpacity': 0.0}

# Add each shape to the map with its index in properties
for idx, row in gdf.iterrows():
    # Add the appropriate tooltip based on whether it's min, max, or neither
    if idx == min_index_ki_tk:
        tooltip = f"Min Value: {min_ki_tk}"
    elif idx == max_index_ki_tk:
        tooltip = f"Max Value: {max_ki_tk}"
    else:
        tooltip = None  # No tooltip for other shapes

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}  # Add the index to properties
        },
        style_function=style_function,
        tooltip=tooltip  # Attach the tooltip with min/max value
    )
    geojson.add_to(m)

# Add an explanation for the colors on the map
folium.Marker(
    location=[gdf.geometry.centroid.y.mean() + 0.11, gdf.geometry.centroid.x.mean() + 0.1],
    icon=DivIcon(
        icon_size=(80, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 8pt; color: red;">&#9679; Max percentage of out capacity</div>',
    )
).add_to(m)

folium.Marker(
    location=[gdf.geometry.centroid.y.mean() + 0.08, gdf.geometry.centroid.x.mean() + 0.1],
    icon=DivIcon(
        icon_size=(80, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 8pt; color: blue;">&#9679; Min percentage of out capacity</div>',
    )
).add_to(m)

# Display the map
m.save('minmax_arany_ki.html')
m

In [ ]:
m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=10)

# Function to style the shapes
def style_function(feature):
    index = feature['properties']['idx']  # Use 'idx' which we'll add to properties
    if index == min_index_be_tk:
        return {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.7}
    elif index == max_index_be_tk:
        return {'fillColor': 'red', 'color': 'red', 'weight': 2, 'fillOpacity': 0.7}
    else:
        return {'fillColor': 'transparent', 'color': 'black', 'weight': 1, 'fillOpacity': 0.0}

# Add each shape to the map with its index in properties
for idx, row in gdf.iterrows():
    # Add the appropriate tooltip based on whether it's min, max, or neither
    if idx == min_index_be_tk:
        tooltip = f"Min Value: {min_be_tk}"
    elif idx == max_index_be_tk:
        tooltip = f"Max Value: {max_be_tk}"
    else:
        tooltip = None  # No tooltip for other shapes

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}  # Add the index to properties
        },
        style_function=style_function,
        tooltip=tooltip  # Attach the tooltip with min/max value
    )
    geojson.add_to(m)

# Add an explanation for the colors on the map
folium.Marker(
    location=[gdf.geometry.centroid.y.mean() + 0.11, gdf.geometry.centroid.x.mean() + 0.1],
    icon=DivIcon(
        icon_size=(80, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 8pt; color: red;">&#9679; Max percentage of out capacity</div>',
    )
).add_to(m)

folium.Marker(
    location=[gdf.geometry.centroid.y.mean() + 0.08, gdf.geometry.centroid.x.mean() + 0.1],
    icon=DivIcon(
        icon_size=(80, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 8pt; color: blue;">&#9679; Min percentage of out capacity</div>',
    )
).add_to(m)

# Display the map
m.save('minmax_arany_be.html')
m

In [ ]:
# Normalize the values so that 0 maps to white, negatives to blue, and positives to red
norm = mcolors.TwoSlopeNorm(vmin=min_ki_szgk, vcenter=(max_ki_szgk+min_ki_szgk)/2, vmax=max_ki_szgk)

# Use a colormap that transitions from blue (negative) to white (neutral) to red (positive)
cmap = mcolors.LinearSegmentedColormap.from_list("pos_neg_cmap", ['blue', 'white', 'red'])

# Convert the filtered GeoDataFrame to the same CRS as Folium (WGS84 - EPSG:4326)
gdf_filtered = gdf.to_crs(epsg=4326)

# Function to get the color based on the normalized value
def get_color(value):
    norm_value = norm(value)  # Normalize the value with center at 0
    return mcolors.to_hex(cmap(norm_value))  # Convert color to hex for folium

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the value
for idx, row in gdf_filtered.iterrows():
    value = ki_szgk.iloc[idx, 3]  # Get the corresponding value from the DataFrame
    color = get_color(value)  # Get the color based on the value

    tooltip = f"Value: {value}"  # Tooltip to display the value on hover

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value
    )
    geojson.add_to(m)

# Create a colormap legend using branca
colormap = cmb.LinearColormap(
    colors=['blue', 'white', 'red'],
    vmin=min_ki_szgk, vmax=max_ki_szgk,
    caption='Out Value percentage'
)
colormap.add_to(m)

# Save the map to an HTML file and display it
#m.save('map.html')
m

In [ ]:
# Normalize the values to the range [0, 1] for color mapping
norm = plt.Normalize(vmin=min_ki_szgk, vmax=max_ki_szgk)

# Use the 'viridis' colormap for a balanced color gradient
cmap = mcolors.LinearSegmentedColormap.from_list("weight_cmap", ['blue','green','yellow', 'orange', 'red'])

# Convert the filtered GeoDataFrame to the same CRS as Folium (WGS84 - EPSG:4326)
gdf_filtered = gdf.to_crs(epsg=4326)

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the value
for idx, row in gdf_filtered.iterrows():
    value = ki_szgk.iloc[idx, 2]  # Get the corresponding value from the DataFrame
    color = get_color(value)  # Get the color based on the value

    tooltip = f"Value: {value}"  # Tooltip to display the value on hover

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value
    )
    geojson.add_to(m)

# Create a colormap legend using branca
colormap = cmb.LinearColormap(
    colors=['blue','green','yellow', 'orange', 'red'],
    vmin=min_ki_szgk, vmax=max_ki_szgk,
    caption='Out Value'
)
colormap.add_to(m)

# Save the map to an HTML file and display it
#m.save('percent_arany_ki.html')
m

In [ ]:
values = ki_tk.iloc[:, 2]

# Calculate percentiles
percentiles = np.percentile(values, np.arange(0, 101, 10))  # Dividing into 10 percentiles (0-10, 10-20, ..., 90-100)

# Create a blue-to-red colormap
cmap = mcolors.LinearSegmentedColormap.from_list("blue_red_cmap", ["blue","white", "red"])

# Function to assign colors based on the percentile range
def get_percentile_color(value):
    for i in range(len(percentiles) - 1):
        if percentiles[i] <= value < percentiles[i + 1]:
            color = cmap(i / (len(percentiles) - 1))  # Use blue-to-red colormap
            return mcolors.to_hex(color)
    return mcolors.to_hex(cmap(1.0))  # For the top 100th percentile

# Convert the filtered GeoDataFrame to the same CRS as Folium (WGS84 - EPSG:4326)
gdf_filtered = gdf.to_crs(epsg=4326)

# Add percentiles as a new column to the GeoDataFrame for convenience (optional)
gdf_filtered['percentile'] = np.searchsorted(percentiles, values, side='right')

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the percentile
for idx, row in gdf_filtered.iterrows():
    value = values[idx]
    color = get_percentile_color(value)  # Get the color based on the percentile

    tooltip = f"Value: {value}, Percentile: {gdf_filtered['percentile'][idx]}"  # Tooltip with value and percentile

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value and percentile
    )
    geojson.add_to(m)

# Create a colormap legend using branca
colormap = cmb.LinearColormap(
    colors=['blue', 'white', 'red'],
    vmin=percentiles[0], vmax=percentiles[-1],
    caption='Out Percentile Range'
)
colormap.add_to(m)  # Add the colormap to the map

# Display the map
m.save('percent_arany_ki.html')
m

In [ ]:
values = be_tk.iloc[:, 2]

# Calculate percentiles
percentiles = np.percentile(values, np.arange(0, 101, 10))  # Dividing into 10 percentiles (0-10, 10-20, ..., 90-100)

# Create a blue-to-red colormap
cmap = mcolors.LinearSegmentedColormap.from_list("blue_red_cmap", ["blue","white", "red"])

# Function to assign colors based on the percentile range
def get_percentile_color(value):
    for i in range(len(percentiles) - 1):
        if percentiles[i] <= value < percentiles[i + 1]:
            color = cmap(i / (len(percentiles) - 1))  # Use blue-to-red colormap
            return mcolors.to_hex(color)
    return mcolors.to_hex(cmap(1.0))  # For the top 100th percentile

# Convert the filtered GeoDataFrame to the same CRS as Folium (WGS84 - EPSG:4326)
gdf_filtered = gdf.to_crs(epsg=4326)

# Add percentiles as a new column to the GeoDataFrame for convenience (optional)
gdf_filtered['percentile'] = np.searchsorted(percentiles, values, side='right')

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the percentile
for idx, row in gdf_filtered.iterrows():
    value = values[idx]
    color = get_percentile_color(value)  # Get the color based on the percentile

    tooltip = f"Value: {value}, Percentile: {gdf_filtered['percentile'][idx]}"  # Tooltip with value and percentile

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value and percentile
    )
    geojson.add_to(m)

# Create a colormap legend using branca
colormap = cmb.LinearColormap(
    colors=['blue', 'white', 'red'],
    vmin=percentiles[0], vmax=percentiles[-1],
    caption='Out Percentile Range'
)
colormap.add_to(m)  # Add the colormap to the map

# Display the map
m.save('percent_arany_be.html')
m

In [ ]:
values = ki_tk.iloc[:, 3]

# Reset index of gdf_filtered to ensure it matches with the values
#gdf_filtered = gdf_filtered.iloc[:-1].reset_index(drop=True)  # Also drop the last row in gdf_filtered

# Define color ranges and corresponding intervals
def get_color(value):
    if value < 1:
        return '#00FF00'  # Green
    elif value == 1:
        return '#FFFF00'  # Yellow
    else:
        return '#FF0000'  # Red

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the value interval
for idx, row in gdf_filtered.iterrows():
    value = values.iloc[idx]  # Get the value
    color = get_color(value)  # Get the color based on the defined intervals and conditions

    tooltip = f"Value: {value}"  # Tooltip with value

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value
    )
    geojson.add_to(m)

# Display the map
m.save('szines_arany_elotte_ki.html')
m

In [ ]:
values = be_tk.iloc[:, 3]

# Reset index of gdf_filtered to ensure it matches with the values
#gdf_filtered = gdf_filtered.iloc[:-1].reset_index(drop=True)  # Also drop the last row in gdf_filtered

# Define color ranges and corresponding intervals
def get_color(value):
    if value < 1:
        return '#00FF00'  # Green
    elif value == 1:
        return '#FFFF00'  # Yellow
    else:
        return '#FF0000'  # Red

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the value interval
for idx, row in gdf_filtered.iterrows():
    value = values.iloc[idx]  # Get the value
    color = get_color(value)  # Get the color based on the defined intervals and conditions

    tooltip = f"Value: {value}"  # Tooltip with value

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value
    )
    geojson.add_to(m)

# Display the map
m.save('szines_arany_elotte_be.html')
m

In [ ]:
values =  (ki_tk.iloc[:, 0] + ki_szgk.iloc[:, 0]) / ki_tk.iloc[:, 1]

# Reset index of gdf_filtered to ensure it matches with the values
#gdf_filtered = gdf_filtered.iloc[:-1].reset_index(drop=True)  # Also drop the last row in gdf_filtered

# Define color ranges and corresponding intervals
def get_color(value):
    if value < 1:
        return '#00FF00'  # Green
    elif value == 1:
        return '#FFFF00'  # Yellow
    else:
        return '#FF0000'  # Red

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the value interval
for idx, row in gdf_filtered.iterrows():
    value = values.iloc[idx]  # Get the value
    color = get_color(value)  # Get the color based on the defined intervals and conditions

    tooltip = f"Value: {value}"  # Tooltip with value

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value
    )
    geojson.add_to(m)

# Display the map
m.save('szines_arany_utana_ki.html')
m

In [ ]:
values =  (be_tk.iloc[:, 0] + be_szgk.iloc[:, 0]) / be_tk.iloc[:, 1]

# Reset index of gdf_filtered to ensure it matches with the values
#gdf_filtered = gdf_filtered.iloc[:-1].reset_index(drop=True)  # Also drop the last row in gdf_filtered

# Define color ranges and corresponding intervals
def get_color(value):
    if value < 1:
        return '#00FF00'  # Green
    elif value == 1:
        return '#FFFF00'  # Yellow
    else:
        return '#FF0000'  # Red

# Create a base map
m = folium.Map(location=[gdf_filtered.geometry.centroid.y.mean(), gdf_filtered.geometry.centroid.x.mean()], zoom_start=10)

# Add each shape to the map with its color based on the value interval
for idx, row in gdf_filtered.iterrows():
    value = values.iloc[idx]  # Get the value
    color = get_color(value)  # Get the color based on the defined intervals and conditions

    tooltip = f"Value: {value}"  # Tooltip with value

    geojson = folium.GeoJson(
        data={
            'type': 'Feature',
            'geometry': mapping(row['geometry']),
            'properties': {'idx': idx}
        },
        style_function=lambda x, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=tooltip  # Attach the tooltip with the value
    )
    geojson.add_to(m)

# Display the map
m.save('szines_arany_utana_be.html')
m

###Every_OD

In [ ]:
eredeti_OD_tk = df.iloc[2:924, 3:925].reset_index(drop=True)
eredeti_OD_tk = eredeti_OD_tk / 10 # traffic in rush hours
eredeti_OD_tk

eredeti_OD_szgk = df2.iloc[2:924, 3:925].reset_index(drop=True)
eredeti_OD_szgk = eredeti_OD_szgk * 1.5 / 10 # average number of people in a car
eredeti_OD_szgk

igeny_OD = eredeti_OD_tk + eredeti_OD_szgk
igeny_OD.reset_index(drop=True)

In [ ]:
new_index = np.arange(0, 922)
#igeny_OD.columns = new_index
eredeti_OD_tk.columns = new_index
#igeny_OD

In [ ]:
def dijkstra_max_capacity(G, source):
    """Modified Dijkstra's algorithm to find paths with the maximum minimum edge weight, ensuring positive capacity."""
    # Priority queue: stores (-capacity, path length, current node, path)
    queue = [(-float('inf'), 0, source, [])]
    visited = {source: (-float('inf'), 0)}  # Stores the best (capacity, path length) found so far
    paths = {}  # Store the path to reach each node

    while queue:
        # Pop the node with the highest capacity path
        max_cap, length, node, path = heapq.heappop(queue)
        max_cap = -max_cap
        current_path = path + [node]

        if node in paths:
            continue

        paths[node] = current_path

        # Explore neighbors
        for neighbor in G.neighbors(node):
            edge_weight = G[node][neighbor]['weight']

            # Skip edges with non-positive capacity
            if edge_weight <= 0:
                continue

            # The capacity of the new path is the min of the current path capacity and the new edge weight
            new_cap = min(max_cap, edge_weight)
            new_length = length + 1

            # If this path to neighbor has better capacity or is shorter with the same capacity
            if neighbor not in visited or (new_cap > visited[neighbor][0]) or (new_cap == visited[neighbor][0] and new_length < visited[neighbor][1]):
                visited[neighbor] = (new_cap, new_length)
                heapq.heappush(queue, (-new_cap, new_length, neighbor, current_path))

    return paths, visited

In [ ]:
# Initialize node_capacity_dict
node_capacity_dict = {node: 0 for node in G4.nodes()}  # Initialize capacity counts for each node

# List to store the final results
all_routes = []

# Get all nodes in the graph
nodes = list(G4.nodes())

# Iterate over all pairs of nodes
for start_node in nodes:
    # Run the modified Dijkstra's algorithm from the start node
    paths, capacities = dijkstra_max_capacity(G4, start_node)

    # Store the result for each end node
    for end_node in nodes:
        if end_node != start_node:
            # First, consider the first and last nodes
            if end_node in capacities:
                # Path exists between start_node and end_node
                path = paths[end_node]

                # Get the first and last node in the path
                first_node = path[0]
                last_node = path[-1]

                # Get first_zone and last_zone using node_shape_dict
                try:
                    first_zone = node_shape_dict[first_node]
                    last_zone = node_shape_dict[last_node]

                    # Fetch the OD value for this zone pair using a try-except block
                    try:
                        #od_value = igeny_OD.at[first_zone, last_zone]
                        od_value = eredeti_OD_tk.at[first_zone, last_zone]
                    except KeyError:
                        od_value = 0  # Handle the case where the key is not found
                        #print(f"KeyError: Invalid zone combination: first_zone={first_zone}, last_zone={last_zone}")

                    # Increment the capacity in node_capacity_dict for each node in the path
                    for node in path:
                        node_capacity_dict[node] += od_value

                    # Get the max capacity for this path
                    max_capacity = capacities[end_node][0]
                    all_routes.append([first_node, last_node, max_capacity])

                except KeyError:
                    #print(f"KeyError: Invalid node in node_shape_dict for first_node={first_node} or last_node={last_node}")
                    continue

            else:
                # Handle the case where there is no path from start_node to end_node
                max_capacity = 0  # or some other default value indicating no path
                all_routes.append([start_node, end_node, max_capacity])

In [ ]:
print(node_capacity_dict)

In [ ]:
# Calculate outgoing and incoming weights
outgoing_weights = {}
incoming_weights = {}

for node in G4.nodes():
    outgoing_weights[node] = sum(data['weight'] for _, _, data in G4.out_edges(node, data=True))
    incoming_weights[node] = sum(data['weight'] for _, _, data in G4.in_edges(node, data=True))
print(outgoing_weights)

In [ ]:
# Normalize the weights for visualization
max_weight = max(outgoing_weights.values())
normalized_weights = {node: weight / max_weight for node, weight in outgoing_weights.items()}
print(normalized_weights)

In [ ]:
# Create GeoDataFrame for outgoing weights
outgoing_gdf = gpd.GeoDataFrame(
    [{'geometry': Point(node), 'weight': weight} for node, weight in normalized_weights.items()],
    crs="EPSG:4326"
)

# Create GeoDataFrame for incoming weights
incoming_gdf = gpd.GeoDataFrame(
    [{'geometry': Point(node), 'weight': weight} for node, weight in incoming_weights.items()],
    crs="EPSG:4326"
)
#outgoing_gdf.head(3)
#incoming_gdf.head(3)

In [ ]:
# Initialize map centered on Budapest
m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)

# Define a function to choose color based on normalized weight
def get_color(weight):
    cmap = mcolors.LinearSegmentedColormap.from_list("weight_cmap", ['blue','green','yellow', 'orange', 'red'])
    return mcolors.to_hex(cmap(weight))

# Determine the maximum and minimum of the normalized weights
max_weight = max(normalized_weights.values())
min_weight = min(normalized_weights.values())

# Add nodes with sizes and colors based on normalized outgoing weights
for node, normalized_weight in normalized_weights.items():
    # Normalize size by mapping weights linearly from the smallest size to the largest
    size_factor = 10  # Base size factor, adjust to scale all circles appropriately
    radius = size_factor * (normalized_weight - min_weight) / (max_weight - min_weight) + 5

    folium.CircleMarker(
        location=node,
        radius=radius,
        color=get_color(normalized_weight),
        fill=True,
        fill_color=get_color(normalized_weight),
        fill_opacity=0.6,
        popup=f'Outgoing weight: {outgoing_weights[node]} (Normalized: {normalized_weight:.2f})'
    ).add_to(m)

# Add color map to the folium map
colormap = folium.StepColormap(
    colors=['blue', 'green', 'yellow', 'orange', 'red'],
    vmin=0, vmax=1,
    index=[0, 0.25, 0.5, 0.75, 1],
    caption='Heatmap Intensity'
)
m.add_child(colormap)
#m.save('buborek.html')
# Display the map
m

In [ ]:
difference_dict = {key: outgoing_weights[key] - node_capacity_dict[key] for key in outgoing_weights}
max_weight = max(outgoing_weights.values())
normalized_weights2 = {node: weight / max_weight for node, weight in difference_dict.items()}
outgoing2_gdf = gpd.GeoDataFrame(
    [{'geometry': Point(node), 'weight': weight} for node, weight in normalized_weights2.items()],
    crs="EPSG:4326"
)

In [ ]:
print(difference_dict)

In [ ]:
# Initialize the map centered on Budapest
m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)

# Define a function to choose color based on the weight
def get_color(value):
    if value > 0:
        return '#00FF00'  # Green for positive values
    else:
        return '#FF0000'  # Red for negative values

# Get the minimum and maximum values of the normalized weights
min_weight = min(normalized_weights2.values())
max_weight = max(normalized_weights2.values())

# Create a custom colormap with red for negative and green for positive values
# Here we scale the colormap so that 0 is the midpoint (transition between red and green)
colormap = cmb.LinearColormap(
    colors=['#FF0000', '#FFFFFF', '#00FF00'],  # Red -> White -> Green
    vmin=min_weight,  # Minimum value is the minimum weight
    vmax=max_weight,  # Maximum value is the maximum weight
).to_step(n=2)  # You can adjust the number of steps for smoother transitions

# Add nodes with colors based on normalized outgoing weights
for node, normalized_weight in normalized_weights2.items():
    folium.CircleMarker(
        location=node,
        radius=8,  # Adjust the size of the markers as needed
        color=get_color(normalized_weight),  # Use the color based on positive/negative
        fill=True,
        fill_color=get_color(normalized_weight),  # Fill color based on the same logic
        fill_opacity=0.6,
    ).add_to(m)

# Add the colormap to the map
colormap.add_to(m)

# Display the map
m